In [51]:
import requests

url = "https://api.twelvedata.com/time_series"

querystring = {"symbol":"AAPL","interval":"15min","outputsize":"5000","apikey":"7ec4dde9d1ca477085a63760b9d9d907"}

headers = {
    'cache-control': "no-cache",
    'postman-token': "3c6fdbb0-a011-8ec5-ad77-9f5a32a56b36"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [52]:
import json

In [53]:
y = json.loads(response.text)

In [54]:
import pandas as pd
tmp = pd.DataFrame(y["values"])
tmp.head()

,datetime,open,high,low,close,volume
0,2020-11-24 15:45:00,115.52500,115.68000,114.97000,115.17000,6519014
1,2020-11-24 15:30:00,115.67000,115.73000,115.36000,115.52080,3009603
2,2020-11-24 15:15:00,115.49000,115.71000,115.46010,115.67500,2623868
3,2020-11-24 15:00:00,115.36000,115.50000,115.17000,115.48000,2174229
4,2020-11-24 14:45:00,115.13000,115.37960,115.00000,115.36170,2031637


In [55]:

tmp["candle"] = 1

tmp["length"] = 1

tmp["shadow_1"] = 1
tmp["shadow_2"] = 1
tmp["strategy"] = 1
tmp["buy/sell/hold"] = 1

In [56]:
# Creation of candle length

for i in range(len(tmp)):
    if tmp.iloc[i,1] > tmp.iloc[i,4]:
        tmp.iloc[i, 6] = "red"
        tmp.iloc[i, 7] = float(tmp.iloc[i,1]) - float(tmp.iloc[i,3])
    else:
        tmp.iloc[i, 6] = "green"
        tmp.iloc[i, 7] = float(tmp.iloc[i,3]) - float(tmp.iloc[i,1])

In [57]:
# creation of shadows

for i in range(len(tmp)):
    tmp.iloc[i, 8] = float(tmp.iloc[i,2]) - float(tmp.iloc[i,1])
    tmp.iloc[i, 9] = float(tmp.iloc[i,4]) - float(tmp.iloc[i,3])

In [58]:
# Defining Support and Resistance
try:
    tmp.iloc[0, 10] = "support"
    support = tmp.iloc[0, 1]
    resistance = tmp.iloc[0, 1]
    for i in range(len(tmp)):
        if tmp.iloc[i+1, 1] > support:
            if tmp.iloc[i+1, 1] >= resistance:
                resistance = tmp.iloc[i+1, 1]
                tmp.iloc[i+1, 10] = "resistance"
            else:
                tmp.iloc[i+1, 10] = "tie-breaker"
        else:
            support = tmp.iloc[i+1, 1]
            tmp.iloc[i+1, 10] = "support"
except IndexError as error:
    pass

In [59]:
# Making buy, sell and hold strategy

try:
    for i in range(len(tmp)):
        if tmp.iloc[i, 10] == "support" and tmp.iloc[i+1, 10] == "support":
            tmp.iloc[i, 11] = "hold"
        if tmp.iloc[i, 10] == "resistance" and tmp.iloc[i+1, 10] == "resistance":
            tmp.iloc[i, 11] = "hold"
except IndexError as error:
    pass

In [60]:
try: 
    for i in range(len(tmp)):
        if tmp.iloc[i, 10] == "resistance" and tmp.iloc[i+1, 10] == "tie-breaker":
            tmp.iloc[i, 11] = "sell"
        elif tmp.iloc[i, 10] == "tie-breaker" and tmp.iloc[i+1, 10] == "tie-breaker":
            tmp.iloc[i, 11] = "sell"
        elif tmp.iloc[i, 10] == "tie-breaker" and tmp.iloc[i+1, 10] == "resistance":
            tmp.iloc[i, 11] = "buy" 
        elif tmp.iloc[i, 10] == "support" and tmp.iloc[i+1, 10] == "resistance":
            tmp.iloc[i, 11] = "buy"
        elif tmp.iloc[i, 10] == "tie-breaker" and tmp.iloc[i+1, 10] == "support":
            tmp.iloc[i, 11] = "sell"
        elif tmp.iloc[i, 10] == "support" and tmp.iloc[i+1, 10] == "tie-breaker":
            tmp.iloc[i, 11] = "buy"

except IndexError as error:
    pass

# Adding Bollinger bands

In [62]:
tmp["middle_band"] = tmp['open'].rolling(window=20).mean()

tmp['upper_band'] = tmp['middle_band'] + 1.96*tmp['close'].rolling(window=20).std()

tmp['lower_band'] = tmp['middle_band'] - 1.96*tmp['close'].rolling(window=20).std()

In [63]:
tmp['middle_band'].fillna(0, inplace=True)
tmp['upper_band'].fillna(0, inplace=True)
tmp['lower_band'].fillna(0, inplace=True)

In [64]:
tmp["buy/sell/bb"] = 1

In [65]:
for i in range(len(tmp)):
    if float(tmp.iloc[i,1]) > float(tmp.iloc[i,13]):
        tmp.iloc[i, 15] = "sell"
    elif float(tmp.iloc[i,1]) < float(tmp.iloc[i,14]):
        tmp.iloc[i, 15] = "buy"
    else:
        tmp.iloc[i, 15] = "hold"

# Adding RSI

In [66]:
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

In [67]:
import numpy as np
tmp["rsi"] = RSI(tmp['open'].astype(float), 20)

In [68]:
tmp['rsi'].fillna(0, inplace=True)

In [69]:
tmp["buy/sell/rsi"] = 1

In [70]:
for i in range(len(tmp)):
    if float(tmp.iloc[i,16]) == 0:
        tmp.iloc[i, 17] = "dont_count"
    elif float(tmp.iloc[i,16]) >= 70:
        tmp.iloc[i, 17] = "sell"
    elif float(tmp.iloc[i,16]) >= 70:
        tmp.iloc[i, 17] = "buy"
    else:
        tmp.iloc[i, 17] = "hold"

In [71]:
tmp.head(50)

,datetime,open,high,low,close,volume,candle,length,shadow_1,shadow_2,strategy,buy/sell/hold,middle_band,upper_band,lower_band,buy/sell/bb,rsi,buy/sell/rsi
0,2020-11-24 15:45:00,115.52500,115.68000,114.97000,115.17000,6519014,red,0.5550,0.1550,0.2000,support,buy,0.000000,0.000000,0.000000,sell,0.000000,dont_count
1,2020-11-24 15:30:00,115.67000,115.73000,115.36000,115.52080,3009603,red,0.3100,0.0600,0.1608,resistance,1,0.000000,0.000000,0.000000,sell,0.000000,dont_count
2,2020-11-24 15:15:00,115.49000,115.71000,115.46010,115.67500,2623868,green,-0.0299,0.2200,0.2149,support,hold,0.000000,0.000000,0.000000,sell,0.000000,dont_count
3,2020-11-24 15:00:00,115.36000,115.50000,115.17000,115.48000,2174229,green,-0.1900,0.1400,0.3100,support,hold,0.000000,0.000000,0.000000,sell,0.000000,dont_count
4,2020-11-24 14:45:00,115.13000,115.37960,115.00000,115.36170,2031637,green,-0.1300,0.2496,0.3617,support,hold,0.000000,0.000000,0.000000,sell,0.000000,dont_count
5,2020-11-24 14:30:00,115.08000,115.27500,115.02000,115.13000,2272384,green,-0.0600,0.1950,0.1100,support,buy,0.000000,0.000000,0.000000,sell,0.000000,dont_count
6,2020-11-24 14:15:00,115.47270,115.47500,114.97500,115.07500,2971396,red,0.4977,0.0023,0.1000,tie-breaker,buy,0.000000,0.000000,0.000000,sell,0.000000,dont_count
7,2020-11-24 14:00:00,115.69500,115.85000,115.31500,115.47500,3216697,red,0.3800,0.1550,0.1600,resistance,sell,0.000000,0.000000,0.000000,sell,0.000000,dont_count
8,2020-11-24 13:45:00,115.47500,115.70000,115.33000,115.69010,2506402,green,-0.1450,0.2250,0.3601,tie-breaker,sell,0.000000,0.000000,0.000000,sell,0.000000,dont_count
9,2020-11-24 13:30:00,115.05000,115.55500,115.04000,115.49000,3360681,green,-0.0100,0.5050,0.4500,support,hold,0.000000,0.000000,0.000000,sell,0.000000,dont_count


In [72]:
tmp_subset = tmp[["open", "strategy","candle","length","shadow_1","shadow_2","buy/sell/hold","buy/sell/bb", "buy/sell/rsi"]]

tmp_subset = tmp_subset[tmp_subset["buy/sell/rsi"] != "dont_count"]

In [49]:
tmp_subset[(tmp_subset["buy/sell/rsi"] == "buy") & (tmp_subset["buy/sell/bb"] == "buy") & (tmp_subset["buy/sell/hold"] == "buy")]

,open,strategy,candle,length,shadow_1,shadow_2,buy/sell/hold,buy/sell/bb,buy/sell/rsi


In [73]:
tmp_subset.head(50)

,open,strategy,candle,length,shadow_1,shadow_2,buy/sell/hold,buy/sell/bb,buy/sell/rsi
20,114.24500,support,green,-0.1150,0.6600,0.7727,hold,buy,hold
21,113.40080,support,green,-0.0595,0.8492,0.9049,buy,buy,hold
22,113.68000,tie-breaker,red,0.2950,0.1100,0.0200,sell,buy,hold
23,113.02000,support,green,-0.0050,0.8000,0.6685,buy,buy,hold
24,113.11000,tie-breaker,red,0.5200,0.0000,0.4300,sell,buy,hold
25,113.89000,tie-breaker,red,1.1250,0.1500,0.3450,sell,hold,hold
26,113.99000,tie-breaker,red,0.2400,0.1900,0.1000,sell,hold,hold
27,114.04000,tie-breaker,red,0.2800,0.0400,0.2300,sell,hold,hold
28,114.34000,tie-breaker,red,0.3500,0.0050,0.0450,sell,hold,hold
29,114.20000,tie-breaker,green,-0.1500,0.2700,0.2850,sell,hold,hold
